In [3]:
import mne
import os.path as op
from tools import files
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [4]:
# functions
trim = lambda x: trim_mean(x, 0.1, axis=0)

In [5]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
meg_path = op.join(main_path, "MEG")

In [8]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participant = participants[0]
pp_path = op.join(meg_path, participant, "new_v1")

True

In [9]:
results_path = op.join(main_path, "RESULTS", "WHATEVER_SOURCE_SPACE")
files.make_folder(results_path)

In [ ]:
all_epochs = files.get_files(pp_path, "epochs-TD", "-epo.fif")[2]
fwd_solution =